<a href="https://colab.research.google.com/github/ews46167-art/mgmt467-analytics-portfolio/blob/main/Lab_VertexAI_BigQuery_PromptsOnly_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab: Vertex AI–Assisted BigQuery Analytics — Example Prompts
**Goal:** Practice moving from simple SQL to complex analytics in BigQuery using *only* carefully engineered prompts with Vertex AI (Gemini).  
**Important:** This notebook contains **prompts only** (no starter code). Paste the prompts into **Vertex AI Studio**, **Vertex AI in Colab Enterprise**, or your chosen chat interface, and then run the generated SQL directly in **BigQuery**. If you decide to automate later, you can ask Vertex AI to convert the winning SQL into a Colab pipeline.

## How to use this prompts-only notebook
1. Open **Vertex AI Studio** (or Gemini in Colab Enterprise chat panel).  
2. Copy a prompt from this notebook and paste it into the model. Do **not** paste any code from here; let the model generate it.  
3. Run the generated SQL in **BigQuery** (Console → BigQuery Studio).  
4. Iterate: refine the prompt when results aren’t what you expect.  
5. Document: capture your final SQL, plus a one-sentence takeaway, in your notes/README.

## Dataset assumptions
Use one of these sources (adjust table paths accordingly):
- **Global Superstore (Kaggle)** loaded into BigQuery (e.g., `[YOUR_PROJECT].superstore_data.sales`)  
- **TheLook eCommerce** public dataset: `bigquery-public-data.thelook_ecommerce`  
If you are using *Global Superstore*, make sure column names match your schema (e.g., `Order_Date`, `Region`, `Category`, `Sub_Category`, `Sales`, `Profit`, `Discount`, `State`, `Customer_ID`, `Ship_Mode`).

---
## Prompting guardrails (quick checklist)
- **Be explicit**: table path, column names, filters, output columns, sort order, and limits.  
- **Ask for runnable SQL**: “Return a BigQuery SQL block only.”  
- **Control cost**: ask for `LIMIT` during exploration and remove it for the final run.  
- **Validate**: request a brief explanation of why each clause is present and how you can sanity-check results.
---

## Install Dependencies

In [ ]:
# Install the Google Cloud BigQuery client library
!pip install google-cloud-bigquery==3.17.0 pandas==2.1.4

# Authenticate your Colab environment
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.2/230.2 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 22.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
  Attempting uninstall: google-cloud-bigquery
    Found existing installation: google-cloud-bigquery 3.38.0
    Uninstalling google-cloud-bigquery-3.38.0:
      Successfully uninstalled google-cloud-bigquery-3.38.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency con

## Copy Schema to a dataframe

In [2]:
from google.cloud import bigquery
import pandas as pd

# Replace with your Google Cloud Project ID
project_id = 'manifest-chain-471119-t8' # This is derived from your provided table name
dataset_id = 'lab1_foundation'
table_id = 'superstore'

# Construct a BigQuery client object.
client = bigquery.Client(project=project_id)

# Get the table object
table_ref = client.dataset(dataset_id).table(table_id)
table = client.get_table(table_ref)

# Extract schema information
schema_list = []
for field in table.schema:
    schema_list.append({
        'name': field.name,
        'field_type': field.field_type,
        'mode': field.mode,
        'description': field.description
    })

# Convert to Pandas DataFrame
schema_df = pd.DataFrame(schema_list)

# Display the schema DataFrame (optional, for verification)
print("Schema DataFrame created:")
# To see the output, run the code.


Schema DataFrame created:


## CLean Column Names

In [3]:
# --- 1. Clean the Column Names ---
# Create a 'clean_name' column with standard naming conventions:
# lowercase, with spaces and hyphens replaced by underscores.
schema_df['clean_name'] = schema_df['name'].str.lower().str.replace(' ', '_').str.replace('-', '_')


# --- 2. Generate the Aliases for the SELECT Clause ---
column_expressions = []
for index, row in schema_df.iterrows():
    original_name = row['name']
    clean_name = row['clean_name']

    # If the original name contains a space or special character, it needs to be
    # enclosed in backticks (`) in the SQL statement.
    if ' ' in original_name or '-' in original_name:
        expression = f'`{original_name}` AS {clean_name}'
    else:
        # If the name is already clean, we still alias it for consistency.
        expression = f'{original_name} AS {clean_name}'
    column_expressions.append(expression)

# Join all the individual column expressions into a single, formatted string.
select_clause = ",\n  ".join(column_expressions)


# --- 3. Construct the Final CREATE VIEW Statement ---
new_view_id = 'superstore_clean' # You can change this if you like

create_view_sql = f"""
CREATE OR REPLACE VIEW `{project_id}.{dataset_id}.{new_view_id}` AS
SELECT
  {select_clause}
FROM
  `{project_id}.{dataset_id}.{table_id}`;
"""

# --- 4. Print the Final SQL ---
print("--- Copy the SQL below and run it in your BigQuery Console ---")
print(create_view_sql)

--- Copy the SQL below and run it in your BigQuery Console ---

CREATE OR REPLACE VIEW `manifest-chain-471119-t8.lab1_foundation.superstore_clean` AS
SELECT
  `Row ID` AS row_id,
  `Order ID` AS order_id,
  `Order Date` AS order_date,
  `Ship Date` AS ship_date,
  `Ship Mode` AS ship_mode,
  `Customer ID` AS customer_id,
  `Customer Name` AS customer_name,
  Segment AS segment,
  Country AS country,
  City AS city,
  State AS state,
  `Postal Code` AS postal_code,
  Region AS region,
  `Product ID` AS product_id,
  Category AS category,
  `Sub-Category` AS sub_category,
  `Product Name` AS product_name,
  Sales AS sales,
  Quantity AS quantity,
  Discount AS discount,
  Profit AS profit
FROM
  `manifest-chain-471119-t8.lab1_foundation.superstore`;



## Generate View with standard column naming convention

In [4]:
# Execute the CREATE VIEW SQL query
try:
    query_job = client.query(create_view_sql)  # API request
    query_job.result()  # Waits for the query to finish
    print(f"View '{new_view_id}' created/replaced successfully in dataset '{dataset_id}'.")
except Exception as e:
    print(f"An error occurred while creating the view: {e}")

# Now, let's print 10 rows from the newly created view to verify
print(f"\n--- First 10 rows from the new view '{new_view_id}' ---")
try:
    # Construct a reference to the new view
    view_table_ref = client.dataset(dataset_id).table(new_view_id)

    # Fetch the first 10 rows
    rows = client.list_rows(view_table_ref, max_results=10)

    # Print header
    print(" | ".join([field.name for field in rows.schema]))
    print("-" * 80) # Separator

    # Print rows
    for row in rows:
        print(" | ".join([str(item) for item in row.values()]))

except Exception as e:
    print(f"An error occurred while fetching rows from the view: {e}")



View 'superstore_clean' created/replaced successfully in dataset 'lab1_foundation'.

--- First 10 rows from the new view 'superstore_clean' ---
row_id | order_id | order_date | ship_date | ship_mode | customer_id | customer_name | segment | country | city | state | postal_code | region | product_id | category | sub_category | product_name | sales | quantity | discount | profit
--------------------------------------------------------------------------------
An error occurred while fetching rows from the view: 400 GET https://bigquery.googleapis.com/bigquery/v2/projects/manifest-chain-471119-t8/datasets/lab1_foundation/tables/superstore_clean/data?maxResults=10&formatOptions.useInt64Timestamp=True&prettyPrint=false: Cannot list a table of type VIEW.


In [6]:
# This assumes your 'client' object from the previous cell is still active
# and correctly authenticated.

print("✅ Step 1: Defining the query string...")

query_string = """
SELECT
  order_id,
  customer_name,
  product_name,
  sales,
  profit
FROM
  `manifest-chain-471119-t8.lab1_foundation.superstore_clean`
LIMIT 10;
"""

print("✅ Step 2: Sending the query to BigQuery. This may take a moment...")

# Use a try-except block to catch potential errors
try:
    query_job = client.query(query_string)

    print("✅ Step 3: Waiting for query to complete and fetching results...")
    results_df = query_job.to_dataframe()

    print(f"✅ Step 4: Query finished. Found {len(results_df)} rows.")

    if results_df.empty:
        print("\n⚠️ The query ran successfully but returned an empty result. Please double-check that your 'superstore_clean' view exists and the original table has data.")
    else:
        print("\n--- Displaying Results ---")
        display(results_df)

except Exception as e:
    print(f"\n❌ An error occurred: {e}")

✅ Step 1: Defining the query string...
✅ Step 2: Sending the query to BigQuery. This may take a moment...
✅ Step 3: Waiting for query to complete and fetching results...
✅ Step 4: Query finished. Found 10 rows.

--- Displaying Results ---


,order_id,customer_name,product_name,sales,profit
0,CA-2015-154900,Sung Shariari,Avery 518,3.15,1.5120
1,CA-2015-154900,Sung Shariari,Adams Telephone Message Book W/Dividers/Space ...,22.72,10.2240
2,US-2016-152415,Patrick O'Donnell,"C-Line Magnetic Cubicle Keepers, Clear Polypro...",14.82,6.2244
3,US-2016-152415,Patrick O'Donnell,"Howard Miller 14-1/2"" Diameter Chrome Round Wa...",191.82,61.3824
4,CA-2016-153269,Pamela Stobb,"Personal Folder Holder, Ebony",11.21,3.3630
5,CA-2016-153269,Pamela Stobb,"Situations Contoured Folding Chairs, 4/Set",354.90,88.7250
6,CA-2016-153269,Pamela Stobb,Xerox 193,17.94,8.7906
7,CA-2016-153269,Pamela Stobb,GBC Binding covers,51.80,23.3100
8,CA-2015-158792,Brian Dahlen,Staples,22.20,10.4340
9,CA-2016-141082,Fred McMath,Avery 517,3.69,1.7343


## Part A — SQL Warm‑Up (SELECT, WHERE, ORDER BY, LIMIT, DISTINCT)
**Aim:** Build confidence with precise, unambiguous prompts that yield clean, runnable SQL.

### A1. Unique values (DISTINCT)
**Prompt (paste in Vertex AI):**
```
Act as a senior BigQuery analyst. Produce a **single runnable BigQuery SQL** (no commentary) for:
- Task: List all unique `Sub_Category` values sold in the 'West' region.
- Table: `manifest-chain-471119-t8.lab1_foundation.superstore`
- Filter: `Region = 'West'`
- Output: a single column named `Sub_Category`
- Sort: alphabetically A→Z
- Add: `LIMIT 100` to control cost during exploration.
```
**Reflection:** Did the result match your expectations? If not, what ambiguity in your prompt might have caused the mismatch?

I asked AI for the query based on the prompt that was given above. At first it just gave me back the exact text that i pasted in so i think it was confused. Then i told it that i wanted an SQL query, and it returned the below query. Maybe the way that the colon was set up in the prompt made it think that i was giving it a query when i was giving it a prompt for the quesry i wanted.

In [7]:
query_string = """
SELECT
    DISTINCT `Sub-Category` AS Sub_Category
FROM
    `manifest-chain-471119-t8.lab1_foundation.superstore_clean`
WHERE
    Region = 'West'
ORDER BY
    Sub_Category ASC
LIMIT 100
"""
results_df = query_job.to_dataframe()
display(results_df)

,order_id,customer_name,product_name,sales,profit
0,CA-2015-154900,Sung Shariari,Avery 518,3.15,1.5120
1,CA-2015-154900,Sung Shariari,Adams Telephone Message Book W/Dividers/Space ...,22.72,10.2240
2,US-2016-152415,Patrick O'Donnell,"C-Line Magnetic Cubicle Keepers, Clear Polypro...",14.82,6.2244
3,US-2016-152415,Patrick O'Donnell,"Howard Miller 14-1/2"" Diameter Chrome Round Wa...",191.82,61.3824
4,CA-2016-153269,Pamela Stobb,"Personal Folder Holder, Ebony",11.21,3.3630
5,CA-2016-153269,Pamela Stobb,"Situations Contoured Folding Chairs, 4/Set",354.90,88.7250
6,CA-2016-153269,Pamela Stobb,Xerox 193,17.94,8.7906
7,CA-2016-153269,Pamela Stobb,GBC Binding covers,51.80,23.3100
8,CA-2015-158792,Brian Dahlen,Staples,22.20,10.4340
9,CA-2016-141082,Fred McMath,Avery 517,3.69,1.7343


### A2. Top‑N by metric (ORDER BY … DESC)
**Prompt:**
```
BigQuery SQL only.
Task: Return the top 10 customers by total profit.
Table: `mgmt-467-47888.lab_foundation.superstore`
Columns used: `Customer_ID`, `Profit`
Output columns: `Customer_ID`, `total_profit`
Logic: SUM Profit per customer, order by `total_profit` DESC
Add `LIMIT 10`.
```
**Tip:** If your schema uses different identifiers (e.g., `Customer Name`), restate column names explicitly.

In [11]:
# This assumes your 'client' object from the previous cell is still active
# and correctly authenticated.

print("✅ Step 1: Defining the query string...")

query_string = """
SELECT
    customer_id,
    SUM(profit) AS total_profit
FROM
    `manifest-chain-471119-t8.lab1_foundation.superstore_clean`
GROUP BY
    customer_id
ORDER BY
    total_profit DESC
LIMIT 10;
"""

print("✅ Step 2: Sending the query to BigQuery. This may take a moment...")

# Use a try-except block to catch potential errors
try:
    query_job = client.query(query_string)

    print("✅ Step 3: Waiting for query to complete and fetching results...")
    results_df2 = query_job.to_dataframe() # Corrected line

    print(f"✅ Step 4: Query finished. Found {len(results_df2)} rows.") # Corrected variable

    if results_df2.empty: # Corrected variable
        print("\n⚠️ The query ran successfully but returned an empty result. Please double-check that your 'superstore_clean' view exists and has data.")
    else:
        print("\n--- Displaying Results ---")
        display(results_df2) # Corrected variable

except Exception as e:
    print(f"\n❌ An error occurred: {e}")

✅ Step 1: Defining the query string...
✅ Step 2: Sending the query to BigQuery. This may take a moment...
✅ Step 3: Waiting for query to complete and fetching results...
✅ Step 4: Query finished. Found 10 rows.

--- Displaying Results ---


,customer_id,total_profit
0,TC-20980,8981.3239
1,RB-19360,6976.0959
2,SC-20095,5757.4119
3,HL-15040,5622.4292
4,AB-10105,5444.8055
5,TA-21385,4703.7883
6,CM-12385,3899.8904
7,KD-16495,3038.6254
8,AR-10540,2884.6208
9,DR-12940,2869.0760


### A3. Basic filtering (WHERE) + sanity checks
**Prompt:**
```
BigQuery SQL only.
Task: Count orders shipped with each `Ship_Mode`, but only for orders in the 'Technology' category.
Table: `[YOUR_PROJECT].superstore_data.sales`
Output: `Ship_Mode`, `order_count`
Logic: COUNT(*) grouped by `Ship_Mode`
Sort by `order_count` DESC
```
**Validation ask:** “Also list two quick sanity checks to verify the numbers.”

**Two Sanity Checks:**
- If you add up all the order numbers from each ship mode, it should be the same as the total orders for the technology category.
- 2nd sanity check

In [12]:
# This assumes your 'client' object from the previous cell is still active
# and correctly authenticated.

print("✅ Step 1: Defining the query string...")

query_string = """
SELECT
    ship_mode,
    COUNT(*) AS order_count
FROM
    `manifest-chain-471119-t8.lab1_foundation.superstore_clean`
WHERE
    category = 'Technology'
GROUP BY
    ship_mode
ORDER BY
    order_count DESC;
"""

print("✅ Step 2: Sending the query to BigQuery. This may take a moment...")

# Use a try-except block to catch potential errors
try:
    query_job = client.query(query_string)

    print("✅ Step 3: Waiting for query to complete and fetching results...")
    results_df3 = query_job.to_dataframe() # Corrected line

    print(f"✅ Step 4: Query finished. Found {len(results_df3)} rows.") # Corrected variable

    if results_df3.empty: # Corrected variable
        print("\n⚠️ The query ran successfully but returned an empty result. Please double-check that your 'superstore_clean' view exists and has data.")
    else:
        print("\n--- Displaying Results ---")
        display(results_df3) # Corrected variable

except Exception as e:
    print(f"\n❌ An error occurred: {e}")

✅ Step 1: Defining the query string...
✅ Step 2: Sending the query to BigQuery. This may take a moment...
✅ Step 3: Waiting for query to complete and fetching results...
✅ Step 4: Query finished. Found 4 rows.

--- Displaying Results ---


,ship_mode,order_count
0,Standard Class,1082
1,Second Class,366
2,First Class,301
3,Same Day,98


## Part B — Grouped Analytics (GROUP BY, HAVING)
**Aim:** Turn raw facts into grouped metrics and filtered aggregations.

### B1. KPI aggregation with WHERE + GROUP BY
**Prompt:**
```
BigQuery SQL only.
Task: Compute monthly revenue for the last 12 full months.
Table: `[YOUR_PROJECT].superstore_data.sales`
Assume: `Order_Date` is a DATE or TIMESTAMP column named exactly `Order_Date`.
Output: `year_month` (YYYY-MM format), `monthly_revenue`
Logic: Truncate date to month, SUM `Sales`, filter to last 12 full months.
Sort by `year_month` ascending.
Include a `LIMIT` safeguard for exploration.
```

In [16]:
# This assumes your 'client' object from the previous cell is still active
# and correctly authenticated.

query_string = """
SELECT
    FORMAT_DATE('%Y-%m', DATE_TRUNC(order_date, MONTH)) AS year_month,
    SUM(sales) AS monthly_revenue
FROM
    `manifest-chain-471119-t8.lab1_foundation.superstore_clean`
-- WHERE
--     order_date >= DATE_SUB(DATE_TRUNC(CURRENT_DATE(), MONTH), INTERVAL 12 MONTH) -- Commented out to include older data
GROUP BY
    year_month
ORDER BY
    year_month ASC
-- LIMIT 100 -- Uncomment this line to limit results during exploration
;
"""

print("✅ Step 1: Defining the query string...")
print(query_string)

print("✅ Step 2: Sending the query to BigQuery. This may take a moment...")

# Use a try-except block to catch potential errors
try:
    query_job = client.query(query_string)

    print("✅ Step 3: Waiting for query to complete and fetching results...")
    results_df4 = query_job.to_dataframe() # Using results_df4 to avoid variable name conflicts

    print(f"✅ Step 4: Query finished. Found {len(results_df4)} rows.")

    if results_df4.empty:
        print("\n⚠️ The query ran successfully but returned an empty result. Please double-check that your 'superstore_clean' view exists and has data.")
    else:
        print("\n--- Displaying Results ---")
        display(results_df4)

except Exception as e:
    print(f"\n❌ An error occurred: {e}")

✅ Step 1: Defining the query string...

SELECT
    FORMAT_DATE('%Y-%m', DATE_TRUNC(order_date, MONTH)) AS year_month,
    SUM(sales) AS monthly_revenue
FROM
    `manifest-chain-471119-t8.lab1_foundation.superstore_clean`
-- WHERE
--     order_date >= DATE_SUB(DATE_TRUNC(CURRENT_DATE(), MONTH), INTERVAL 12 MONTH) -- Commented out to include older data
GROUP BY
    year_month
ORDER BY
    year_month ASC
-- LIMIT 100 -- Uncomment this line to limit results during exploration
;

✅ Step 2: Sending the query to BigQuery. This may take a moment...
✅ Step 3: Waiting for query to complete and fetching results...
✅ Step 4: Query finished. Found 48 rows.

--- Displaying Results ---


,year_month,monthly_revenue
0,2014-01,14236.8950
1,2014-02,4519.8920
2,2014-03,55691.0090
3,2014-04,28295.3450
4,2014-05,23648.2870
5,2014-06,34595.1276
6,2014-07,33946.3930
7,2014-08,27909.4685
8,2014-09,81777.3508
9,2014-10,31453.3930


### B2. Post‑aggregation filter (HAVING)
**Prompt:**
```
BigQuery SQL only.
Task: Find sub-categories whose total profit over the entire dataset is negative.
Table: `[YOUR_PROJECT].superstore_data.sales`
Output: `Sub_Category`, `total_profit`
Logic: SUM `Profit` GROUP BY `Sub_Category`, HAVING SUM(Profit) < 0
Sort by `total_profit` ASC (most negative first).
```
**Why HAVING?** Ask the model to include a 1-sentence explanation of why HAVING is used instead of WHERE here.

We use HAVING to filter groups based on an aggregated value, like the SUM(profit) for each sub_category, after the GROUP BY clause has aggregated the data.

In [18]:
# This assumes your 'client' object from the previous cell is still active
# and correctly authenticated.

query_string = """
SELECT
    sub_category,
    SUM(profit) AS total_profit
FROM
    `manifest-chain-471119-t8.lab1_foundation.superstore_clean`
GROUP BY
    sub_category
HAVING
    SUM(profit) < 0
ORDER BY
    total_profit ASC;
"""

print("✅ Step 1: Defining the query string...")


print("✅ Step 2: Sending the query to BigQuery. This may take a moment...")

# Use a try-except block to catch potential errors
try:
    query_job = client.query(query_string)

    print("✅ Step 3: Waiting for query to complete and fetching results...")
    results_df5 = query_job.to_dataframe() # Using results_df5 for this query

    print(f"✅ Step 4: Query finished. Found {len(results_df5)} rows.")

    if results_df5.empty:
        print("\n⚠️ The query ran successfully but returned an empty result. This means there are no sub-categories with negative total profit in your dataset.")
    else:
        print("\n--- Displaying Results ---")
        display(results_df5)

except Exception as e:
    print(f"\n❌ An error occurred: {e}")

✅ Step 1: Defining the query string...
✅ Step 2: Sending the query to BigQuery. This may take a moment...
✅ Step 3: Waiting for query to complete and fetching results...
✅ Step 4: Query finished. Found 3 rows.

--- Displaying Results ---


,sub_category,total_profit
0,Tables,-17725.4811
1,Bookcases,-3472.5560
2,Supplies,-1189.0995


## Part C — Joins (dimension enrichment)
**Aim:** Use joins to enhance facts with attributes.

### C1. Join facts to a small dimension
*(If you have a customer or product dimension in your schema, use it. Otherwise, request a synthetic example.)*  
**Prompt:**
```
BigQuery SQL only.
Task: Join the sales table to a product dimension to report `Product_ID`, `Product_Name`, and total sales.
Tables: `[YOUR_PROJECT].superstore_data.sales` as s, `[YOUR_PROJECT].superstore_data.products` as p
Join key: `s.Product_ID = p.Product_ID`
Output: `Product_ID`, `Product_Name`, `total_sales`
Sort by `total_sales` DESC
```
**If you lack a dimension table:** Ask the model how to simulate one temporarily via a CTE.

In [32]:
# This assumes your 'client' object from the previous cell is still active
# and correctly authenticated.

query_string = """
WITH
  product_dimension AS(
    -- Simulate a product dimension table using distinct product information from the sales table
    SELECT DISTINCT
      product_id,
      product_name
    FROM
      `manifest-chain-471119-t8.lab1_foundation.superstore_clean`
  ),
  sales_aggregated AS (
    -- Aggregate sales by product
    SELECT
      product_id,
      SUM(sales) AS total_sales
    FROM
      `manifest-chain-471119-t8.lab1_foundation.superstore_clean`
    GROUP BY
      product_id
  )
SELECT
  p.product_id,
  p.product_name,
  sa.total_sales
FROM
  product_dimension AS p
JOIN
  sales_aggregated AS sa
ON
  p.product_id = sa.product_id
ORDER BY
  sa.total_sales DESC
LIMIT 10; -- Added LIMIT for exploration as per best practices
"""

print("✅ Step 1: Defining the query string...")


print("✅ Step 2: Sending the query to BigQuery. This may take a moment...")

# Use a try-except block to catch potential errors
try:
    query_job = client.query(query_string)

    print("✅ Step 3: Waiting for query to complete and fetching results...")
    results_df6 = query_job.to_dataframe()

    print(f"✅ Step 4: Query finished. Found {len(results_df6)} rows.")

    if results_df6.empty:
        print("\n⚠️ The query ran successfully but returned an empty result.")
    else:
        print("\n--- Displaying Results ---")
        display(results_df6)

except Exception as e:
    print(f"\n❌ An error occurred: {e}")

✅ Step 1: Defining the query string...
✅ Step 2: Sending the query to BigQuery. This may take a moment...
✅ Step 3: Waiting for query to complete and fetching results...
✅ Step 4: Query finished. Found 10 rows.

--- Displaying Results ---


,product_id,product_name,total_sales
0,TEC-CO-10004722,Canon imageCLASS 2200 Advanced Copier,61599.824
1,OFF-BI-10003527,Fellowes PB500 Electric Punch Plastic Comb Bin...,27453.384
2,TEC-MA-10002412,Cisco TelePresence System EX90 Videoconferenci...,22638.480
3,FUR-CH-10002024,HON 5400 Series Task Chairs for Big and Tall,21870.576
4,OFF-BI-10001359,GBC DocuBind TL300 Electric Binding System,19823.479
5,OFF-BI-10000545,GBC Ibimaster 500 Manual ProClick Binding System,19024.500
6,TEC-CO-10001449,Hewlett Packard LaserJet 3310 Copier,18839.686
7,TEC-MA-10001127,HP Designjet T520 Inkjet Large Format Printer ...,18374.895
8,OFF-BI-10004995,GBC DocuBind P400 Electric Binding System,17965.068
9,OFF-SU-10000151,High Speed Automatic Electric Letter Opener,17030.312


## Part D — Common Table Expressions (CTEs)
**Aim:** Make complex logic readable and testable in steps.

### D1. Multi‑step ranking with CTEs
**Prompt:**
```
BigQuery SQL only.
Goal: Within each `Region`, rank states by total sales and return top 3 per region.
Table: `[YOUR_PROJECT].superstore_data.sales`
CTE 1 (`state_sales`): SUM(Sales) by `Region`, `State`
CTE 2 (`ranked_state_sales`): Add `RANK() OVER (PARTITION BY Region ORDER BY total_sales DESC)` as `sales_rank`
Final SELECT: rows where `sales_rank <= 3`
Output columns: `Region`, `State`, `total_sales`, `sales_rank`
Sort: by `Region`, then `sales_rank`
```
**Ask for**: a one-paragraph explanation of each step, then **provide only the final runnable SQL**.

state_sales CTE: This first CTE calculates the total sales for each state within each region. It groups the data by region and state and then sums the sales for each combination.
ranked_state_sales CTE: This second CTE takes the results from the state_sales CTE and ranks the states within each region based on their total_sales. The RANK() OVER (PARTITION BY region ORDER BY total_sales DESC) window function assigns a rank to each state; PARTITION BY region ensures the ranking restarts for each region, and ORDER BY total_sales DESC ranks states with higher sales first.
Final SELECT: The final SELECT statement retrieves the results from the ranked_state_sales CTE and filters them to include only the rows where the sales_rank is less than or equal to 3, effectively giving you the top 3 states by sales for each region. The results are then ordered by region and sales rank for clarity.

In [22]:
# This assumes your 'client' object from the previous cell is still active
# and correctly authenticated.

query_string = """
WITH
  state_sales AS (
    -- CTE 1: Calculate total sales per state within each region
    SELECT
      region,
      state,
      SUM(sales) AS total_sales
    FROM
      `manifest-chain-471119-t8.lab1_foundation.superstore_clean`
    GROUP BY
      region,
      state
  ),
  ranked_state_sales AS (
    -- CTE 2: Rank states by total sales within each region
    SELECT
      region,
      state,
      total_sales,
      RANK() OVER (PARTITION BY region ORDER BY total_sales DESC) AS sales_rank
    FROM
      state_sales
  )
-- Final SELECT: Filter for the top 3 states per region
SELECT
  region,
  state,
  total_sales,
  sales_rank
FROM
  ranked_state_sales
WHERE
  sales_rank <= 3
ORDER BY
  region,
  sales_rank;
"""

print("✅ Step 1: Defining the query string...")


print("✅ Step 2: Sending the query to BigQuery. This may take a moment...")

# Use a try-except block to catch potential errors
try:
    query_job = client.query(query_string)

    print("✅ Step 3: Waiting for query to complete and fetching results...")
    results_df7 = query_job.to_dataframe() # Using results_df7

    print(f"✅ Step 4: Query finished. Found {len(results_df7)} rows.")

    if results_df7.empty:
        print("\n⚠️ The query ran successfully but returned an empty result.")
    else:
        print("\n--- Displaying Results ---")
        display(results_df7)

except Exception as e:
    print(f"\n❌ An error occurred: {e}")

✅ Step 1: Defining the query string...
✅ Step 2: Sending the query to BigQuery. This may take a moment...
✅ Step 3: Waiting for query to complete and fetching results...
✅ Step 4: Query finished. Found 12 rows.

--- Displaying Results ---


,region,state,total_sales,sales_rank
0,Central,Texas,170188.0458,1
1,Central,Illinois,80166.1010,2
2,Central,Michigan,76269.6140,3
3,East,New York,310876.2710,1
4,East,Pennsylvania,116511.9140,2
5,East,Ohio,78258.1360,3
6,South,Florida,89473.7080,1
7,South,Virginia,70636.7200,2
8,South,North Carolina,55603.1640,3
9,West,California,457687.6315,1


### D2. Time‑boxed “most improved” analysis
**Prompt:**
```
BigQuery SQL only.
Goal: Identify the top 5 sub-categories with the largest YoY revenue increase from 2023 to 2024.
Table: `[YOUR_PROJECT].superstore_data.sales`
CTE `yr_sales`: SUM(Sales) by `Sub_Category` and `year` extracted from `Order_Date`
Final: pivot or self-join to compute delta (2024 minus 2023) as `yoy_delta`
Output: `Sub_Category`, `sales_2023`, `sales_2024`, `yoy_delta`
Order by `yoy_delta` DESC
Limit 5
```
**Validation:** Ask the model for two quick failure modes (e.g., missing years) and how to handle them.

In this instance, we handled this by looking for the top 5 sub cataegories over all years. Since we didnt have any instances over 2023 to 2024

In [26]:
# This assumes your 'client' object from the previous cell is still active
# and correctly authenticated.

query_string = """
WITH
  state_sales AS (
    -- Calculate total sales per state within each region
    SELECT
      region,
      state,
      SUM(sales) AS total_sales
    FROM
      `manifest-chain-471119-t8.lab1_foundation.superstore_clean`
    GROUP BY
      region,
      state
  ),
  ranked_state_sales AS (
    -- Rank states by total sales within each region
    SELECT
      region,
      state,
      total_sales,
      RANK() OVER (PARTITION BY region ORDER BY total_sales DESC) AS sales_rank
    FROM
      state_sales
  )
-- Filter for the top 5 states per region
SELECT
  region,
  state,
  total_sales,
  sales_rank
FROM
  ranked_state_sales
WHERE
  sales_rank <= 5
ORDER BY
  region,
  sales_rank;
"""

print("✅ Step 1: Defining the query string...")


print("✅ Step 2: Sending the query to BigQuery. This may take a moment...")

# Use a try-except block to catch potential errors
try:
    query_job = client.query(query_string)

    print("✅ Step 3: Waiting for query to complete and fetching results...")
    results_df9 = query_job.to_dataframe() # Using results_df9

    print(f"✅ Step 4: Query finished. Found {len(results_df9)} rows.")

    if results_df9.empty:
        print("\n⚠️ The query ran successfully but returned an empty result.")
    else:
        print("\n--- Displaying Results ---")
        display(results_df9)

except Exception as e:
    print(f"\n❌ An error occurred: {e}")

✅ Step 1: Defining the query string...
✅ Step 2: Sending the query to BigQuery. This may take a moment...
✅ Step 3: Waiting for query to complete and fetching results...
✅ Step 4: Query finished. Found 20 rows.

--- Displaying Results ---


,region,state,total_sales,sales_rank
0,Central,Texas,170188.0458,1
1,Central,Illinois,80166.1010,2
2,Central,Michigan,76269.6140,3
3,Central,Indiana,53555.3600,4
4,Central,Wisconsin,32114.6100,5
5,East,New York,310876.2710,1
6,East,Pennsylvania,116511.9140,2
7,East,Ohio,78258.1360,3
8,East,New Jersey,35764.3120,4
9,East,Massachusetts,28634.4340,5


## Part E — Window Functions (ROW_NUMBER, RANK, DENSE_RANK, LAG/LEAD, moving averages)
**Aim:** Compare rows across partitions and time; compute trends and ranks without collapsing rows.

### E1. Top product per region (ROW_NUMBER)
**Prompt:**
```
BigQuery SQL only.
Task: For each `Region`, return only the single highest-revenue `Sub_Category`.
Table: `[YOUR_PROJECT].superstore_data.sales`
CTE `subcat_sales`: SUM(Sales) by `Region`, `Sub_Category`
Add `ROW_NUMBER() OVER (PARTITION BY Region ORDER BY total_sales DESC)` as rn
Final: filter `rn = 1`
Output: `Region`, `Sub_Category`, `total_sales`
Sort by `Region`
```
**Why `ROW_NUMBER` instead of `RANK`?** Ask the model to add a 2-sentence contrast.

We use ROW_NUMBER instead of RANK when we only want to return a single row per partition, even if there are ties in the ranking metric. ROW_NUMBER assigns a unique, sequential number to each row within its partition, whereas RANK would give the same rank to tied rows, potentially returning multiple "top" rows if they have the same value.

In [28]:
# This assumes your 'client' object from the previous cell is still active
# and correctly authenticated.

query_string = """
WITH
  subcat_sales AS (
    -- CTE to calculate total sales per sub_category within each region
    SELECT
      region,
      sub_category,
      SUM(sales) AS total_sales
    FROM
      `manifest-chain-471119-t8.lab1_foundation.superstore_clean`
    GROUP BY
      region,
      sub_category
  ),
  ranked_subcat_sales AS (
    -- Rank sub_categories by total sales within each region using ROW_NUMBER
    SELECT
      region,
      sub_category,
      total_sales,
      ROW_NUMBER() OVER (PARTITION BY region ORDER BY total_sales DESC) AS rn
    FROM
      subcat_sales
  )
-- Final SELECT: Filter for the top sub_category (rn = 1) per region
SELECT
  region,
  sub_category,
  total_sales
FROM
  ranked_subcat_sales
WHERE
  rn = 1
ORDER BY
  region;
"""

print("✅ Step 1: Defining the query string...")


print("✅ Step 2: Sending the query to BigQuery. This may take a moment...")

# Use a try-except block to catch potential errors
try:
    query_job = client.query(query_string)

    print("✅ Step 3: Waiting for query to complete and fetching results...")
    results_df10 = query_job.to_dataframe() # Using results_df10

    print(f"✅ Step 4: Query finished. Found {len(results_df10)} rows.")

    if results_df10.empty:
        print("\n⚠️ The query ran successfully but returned an empty result.")
    else:
        print("\n--- Displaying Results ---")
        display(results_df10)

except Exception as e:
    print(f"\n❌ An error occurred: {e}")

✅ Step 1: Defining the query string...
✅ Step 2: Sending the query to BigQuery. This may take a moment...
✅ Step 3: Waiting for query to complete and fetching results...
✅ Step 4: Query finished. Found 4 rows.

--- Displaying Results ---


,region,sub_category,total_sales
0,Central,Chairs,85230.646
1,East,Phones,100614.982
2,South,Phones,58304.438
3,West,Chairs,101781.328


### E2. YoY growth with LAG
**Prompt:**
```
BigQuery SQL only.
Task: Compute year-over-year revenue growth for 'Phones' sub-category.
Table: `[YOUR_PROJECT].superstore_data.sales`
Steps:
- Filter to `Sub_Category = 'Phones'`
- Aggregate yearly revenue using EXTRACT(YEAR FROM Order_Date)
- Add `LAG(yearly_revenue) OVER (ORDER BY year)` as `prev_revenue`
- Compute `yoy_pct = 100.0 * (yearly_revenue - prev_revenue) / prev_revenue`
Output: `year`, `yearly_revenue`, `prev_revenue`, `yoy_pct`
Sort by `year` ASC
```
**Ask for**: a guard against divide-by-zero or NULL previous year.

It says that it used the Safe Divide function, so it will return Null if the divisor is zero or Null.

In [29]:
# This assumes your 'client' object from the previous cell is still active
# and correctly authenticated.

query_string = """
WITH
  yearly_phone_sales AS (
    -- Filter to 'Phones' sub-category and aggregate yearly revenue
    SELECT
      EXTRACT(YEAR FROM order_date) AS order_year,
      SUM(sales) AS yearly_revenue
    FROM
      `manifest-chain-471119-t8.lab1_foundation.superstore_clean`
    WHERE
      sub_category = 'Phones'
    GROUP BY
      order_year
  ),
  yearly_phone_sales_with_lag AS (
    -- Add previous year's revenue using LAG
    SELECT
      order_year,
      yearly_revenue,
      LAG(yearly_revenue) OVER (ORDER BY order_year) AS prev_revenue
    FROM
      yearly_phone_sales
  )
-- Compute YoY percentage growth, handling potential division by zero or null
SELECT
  order_year AS year,
  yearly_revenue,
  prev_revenue,
  SAFE_DIVIDE(100.0 * (yearly_revenue - prev_revenue), prev_revenue) AS yoy_pct -- Using SAFE_DIVIDE to handle division by zero/null
FROM
  yearly_phone_sales_with_lag
ORDER BY
  year ASC;
"""

print("✅ Step 1: Defining the query string...")


print("✅ Step 2: Sending the query to BigQuery. This may take a moment...")

# Use a try-except block to catch potential errors
try:
    query_job = client.query(query_string)

    print("✅ Step 3: Waiting for query to complete and fetching results...")
    results_df11 = query_job.to_dataframe() # Using results_df11

    print(f"✅ Step 4: Query finished. Found {len(results_df11)} rows.")

    if results_df11.empty:
        print("\n⚠️ The query ran successfully but returned an empty result. This might mean there is no sales data for the 'Phones' sub-category.")
    else:
        print("\n--- Displaying Results ---")
        display(results_df11)

except Exception as e:
    print(f"\n❌ An error occurred: {e}")

✅ Step 1: Defining the query string...
✅ Step 2: Sending the query to BigQuery. This may take a moment...
✅ Step 3: Waiting for query to complete and fetching results...
✅ Step 4: Query finished. Found 4 rows.

--- Displaying Results ---


,year,yearly_revenue,prev_revenue,yoy_pct
0,2014,77390.806,NaN,NaN
1,2015,68313.702,77390.806,-11.728918
2,2016,78962.030,68313.702,15.587397
3,2017,105340.516,78962.030,33.406545


### E3. 3‑month moving average (MA)
**Prompt:**
```
BigQuery SQL only.
Task: For the 'Corporate' segment, compute a 3-month moving average of monthly revenue.
Table: `[YOUR_PROJECT].superstore_data.sales`
Steps:
- Derive `month` via DATE_TRUNC(Order_Date, MONTH)
- SUM(Sales) per `month`
- Add `AVG(monthly_revenue) OVER (ORDER BY month ROWS BETWEEN 2 PRECEDING AND CURRENT ROW)` as `ma_3`
Output: `month`, `monthly_revenue`, `ma_3`
Sort by `month` ASC
```
**Tip:** Ask the model to include a 1‑line cost control note (e.g., restrict date range while iterating).

In [30]:
# This assumes your 'client' object from the previous cell is still active
# and correctly authenticated.

query_string = """
WITH
  monthly_corporate_sales AS (
    -- Filter for 'Corporate' segment and aggregate monthly revenue
    SELECT
      DATE_TRUNC(order_date, MONTH) AS order_month,
      SUM(sales) AS monthly_revenue
    FROM
      `manifest-chain-471119-t8.lab1_foundation.superstore_clean`
    WHERE
      segment = 'Corporate'
    GROUP BY
      order_month
  )
-- Compute 3-month moving average
SELECT
  FORMAT_DATE('%Y-%m', order_month) AS month,
  monthly_revenue,
  AVG(monthly_revenue) OVER (ORDER BY order_month ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) AS ma_3
FROM
  monthly_corporate_sales
ORDER BY
  order_month ASC
-- For exploration, restrict the date range to control cost:
-- WHERE order_month >= DATE_SUB(DATE_TRUNC(CURRENT_DATE(), MONTH), INTERVAL 24 MONTH)
;
"""

print("✅ Step 1: Defining the query string...")


print("✅ Step 2: Sending the query to BigQuery. This may take a moment...")

# Use a try-except block to catch potential errors
try:
    query_job = client.query(query_string)

    print("✅ Step 3: Waiting for query to complete and fetching results...")
    results_df12 = query_job.to_dataframe() # Using results_df12

    print(f"✅ Step 4: Query finished. Found {len(results_df12)} rows.")

    if results_df12.empty:
        print("\n⚠️ The query ran successfully but returned an empty result. This might mean there is no sales data for the 'Corporate' segment.")
    else:
        print("\n--- Displaying Results ---")
        display(results_df12)

except Exception as e:
    print(f"\n❌ An error occurred: {e}")

✅ Step 1: Defining the query string...
✅ Step 2: Sending the query to BigQuery. This may take a moment...
✅ Step 3: Waiting for query to complete and fetching results...
✅ Step 4: Query finished. Found 48 rows.

--- Displaying Results ---


,month,monthly_revenue,ma_3
0,2014-01,1701.5280,1701.528000
1,2014-02,1183.6680,1442.598000
2,2014-03,11106.7990,4663.998333
3,2014-04,14131.7290,8807.398667
4,2014-05,9142.0000,11460.176000
5,2014-06,3970.9140,9081.547667
6,2014-07,10032.9880,7715.300667
7,2014-08,7451.7740,7151.892000
8,2014-09,15507.7450,10997.502333
9,2014-10,12637.6780,11865.732333


## Part F — Debugging & Optimization Prompts
**Aim:** Use the model as a rubber duck for error handling and performance.

### F1. Explain the error, propose a fix
**Prompt:**
```
I ran this BigQuery SQL and got an error:
WITH
  product_dimension FOR(
    -- Simulate a product dimension table using distinct product information from the sales table
    SELECT DISTINCT
      product_id,
      product_name
    FROM
      `manifest-chain-471119-t8.lab1_foundation.superstore_clean`
  ),
  sales_aggregated AS (
    -- Aggregate sales by product
    SELECT
      product_id,
      SUM(sales) AS total_sales
    FROM
      `manifest-chain-471119-t8.lab1_foundation.superstore_clean`
    GROUP BY
      product_id
  )
SELECT
  p.product_id,
  p.product_name,
  sa.total_sales
FROM
  product_dimension AS p
JOIN
  sales_aggregated AS sa
ON
  p.product_id = sa.product_id
ORDER BY
  sa.total_sales DESC
LIMIT 10; -- Added LIMIT for exploration as per best practices

An error occurred: 400 Syntax error: Expected keyword AS but got keyword FOR at [3:21]; reason: invalidQuery, location: query, message: Syntax error: Expected keyword AS but got keyword FOR at [3:21]
Act as a BigQuery trouble‑shooter.
1) Identify the root cause.
2) Propose the smallest possible fix.
3) Suggest a quick sanity check query to verify the fix.
Return only the corrected SQL and a 2‑sentence rationale.
```

In [33]:
# Corrected SQL and rationale for debugging exercise

"""
The root cause of the error "Syntax error: Expected keyword AS but got keyword FOR at [3:21]" is that the keyword `FOR` was used instead of the keyword `AS` when defining the Common Table Expression (CTE) named `product_dimension`. In BigQuery SQL, you must use `AS` after the CTE name to define its structure.

The fix is to replace `FOR` with `AS` in the definition of the `product_dimension` CTE.

A quick sanity check query to verify the fix would be to simply run the corrected query again and see if it executes without the syntax error and returns the expected structure (product_id, product_name, total_sales).
"""

corrected_sql = """
WITH
  product_dimension AS (
    -- Simulate a product dimension table using distinct product information from the sales table
    SELECT DISTINCT
      product_id,
      product_name
    FROM
      `manifest-chain-471119-t8.lab1_foundation.superstore_clean`
  ),
  sales_aggregated AS (
    -- Aggregate sales by product
    SELECT
      product_id,
      SUM(sales) AS total_sales
    FROM
      `manifest-chain-419913-t8.lab1_foundation.superstore_clean`
    GROUP BY
      product_id
  )
SELECT
  p.product_id,
  p.product_name,
  sa.total_sales
FROM
  product_dimension AS p
JOIN
  sales_aggregated AS sa
ON
  p.product_id = sa.product_id
ORDER BY
  sa.total_sales DESC
LIMIT 10;
"""

print("Corrected SQL:")
print(corrected_sql)

print("\nRationale:")
print("The error was caused by using 'FOR' instead of 'AS' in the CTE definition. Replacing 'FOR' with 'AS' fixes the syntax error.")

# Optional: You can uncomment the following code to run the corrected query
# try:
#     query_job = client.query(corrected_sql)
#     results_df_fixed = query_job.to_dataframe()
#     print("\n--- Sanity Check Results (first 10 rows) ---")
#     display(results_df_fixed)
# except Exception as e:
#     print(f"\n❌ An error occurred during sanity check: {e}")

Corrected SQL:

WITH
  product_dimension AS (
    -- Simulate a product dimension table using distinct product information from the sales table
    SELECT DISTINCT
      product_id,
      product_name
    FROM
      `manifest-chain-471119-t8.lab1_foundation.superstore_clean`
  ),
  sales_aggregated AS (
    -- Aggregate sales by product
    SELECT
      product_id,
      SUM(sales) AS total_sales
    FROM
      `manifest-chain-419913-t8.lab1_foundation.superstore_clean`
    GROUP BY
      product_id
  )
SELECT
  p.product_id,
  p.product_name,
  sa.total_sales
FROM
  product_dimension AS p
JOIN
  sales_aggregated AS sa
ON
  p.product_id = sa.product_id
ORDER BY
  sa.total_sales DESC
LIMIT 10;


Rationale:
The error was caused by using 'FOR' instead of 'AS' in the CTE definition. Replacing 'FOR' with 'AS' fixes the syntax error.


### F2. Reduce cost / improve speed
**Prompt:**
```
Act as a BigQuery cost optimizer.
Given this query (below), list 3 ways to reduce scanned bytes and improve performance without changing the business logic.
WITH
  product_dimension AS (
    -- Simulate a product dimension table using distinct product information from the sales table
    SELECT DISTINCT
      product_id,
      product_name
    FROM
      `manifest-chain-471119-t8.lab1_foundation.superstore_clean`
  ),
  sales_aggregated AS (
    -- Aggregate sales by product
    SELECT
      product_id,
      SUM(sales) AS total_sales
    FROM
      `manifest-chain-419913-t8.lab1_foundation.superstore_clean`
    GROUP BY
      product_id
  )
SELECT
  p.product_id,
  p.product_name,
  sa.total_sales
FROM
  product_dimension AS p
JOIN
  sales_aggregated AS sa
ON
  p.product_id = sa.product_id
ORDER BY
  sa.total_sales DESC
LIMIT 10;
Prioritize: partition filters, column pruning, pre-aggregations, and temporary results via CTEs.
```

Based on the provided query, here are 3 ways to reduce scanned bytes and improve performance without changing the business logic:

1.  **Column Pruning:** In both CTEs and the final SELECT, explicitly select only the columns you need (`product_id`, `product_name`, `sales`). Avoid `SELECT *`. BigQuery is columnar, so scanning fewer columns reduces scanned bytes. The current query already does a good job of this by selecting specific columns.
2.  **Pre-aggregate if possible:** If this query (or parts of it, like `sales_aggregated`) is run frequently, consider creating a materialized view or a scheduled query to pre-aggregate the sales data by product. This would mean the query only needs to scan the smaller, pre-aggregated table instead of the large `superstore_clean` table every time.
3.  **Partitioning (if applicable):** If the underlying `superstore_clean` table is partitioned (e.g., by date), adding a date filter to the initial scan of that table within the `sales_aggregated` CTE (if your analysis is time-boxed) would significantly reduce the amount of data scanned. Since this query doesn't have a time filter, this specific optimization isn't directly applicable without changing the business logic, but it's a crucial general technique. For this query, ensuring the `superstore_clean` table is clustered by `product_id` could also improve join performance.

## Part G — Validation & Counter‑examples (DIVE: Validate)
**Aim:** Avoid “first‑answer fallacy” by testing alternatives.

### G1. Ask for counter‑queries
**Prompt:**
```
I concluded that 'Tables' is a high‑sales but negative‑profit sub-category due to high discounts.
Create two alternative BigQuery SQL queries that could falsify or nuance this finding:
- One that slices by region and time
- One that controls for order priority or ship mode
Return BigQuery SQL only, then a one-paragraph note on how to compare outcomes.
```

In [35]:
# This assumes your 'client' object is still active and correctly authenticated.

# Query 1: Slice by region and time
query_string_1 = """
SELECT
    region,
    EXTRACT(YEAR FROM order_date) AS order_year,
    SUM(sales) AS total_sales,
    SUM(profit) AS total_profit,
    AVG(discount) AS average_discount
FROM
    `manifest-chain-471119-t8.lab1_foundation.superstore_clean`
WHERE
    sub_category = 'Tables'
GROUP BY
    region,
    order_year
ORDER BY
    region,
    order_year;
"""

print("--- Results for Query 1 (Sliced by Region and Time) ---")
try:
    query_job_1 = client.query(query_string_1)
    results_df_1 = query_job_1.to_dataframe()

    if results_df_1.empty:
        print("\n⚠️ Query 1 ran successfully but returned an empty result. No sales data for 'Tables' found.")
    else:
        display(results_df_1)

except Exception as e:
    print(f"\n❌ An error occurred with Query 1: {e}")

print("\n--- Results for Query 2 (Control for Ship Mode) ---")

# Query 2: Control for ship mode
query_string_2 = """
SELECT
    ship_mode,
    SUM(sales) AS total_sales,
    SUM(profit) AS total_profit,
    AVG(discount) AS average_discount
FROM
    `manifest-chain-471119-t8.lab1_foundation.superstore_clean`
WHERE
    sub_category = 'Tables'
GROUP BY
    ship_mode
ORDER BY
    total_profit ASC;
"""

try:
    query_job_2 = client.query(query_string_2)
    results_df_2 = query_job_2.to_dataframe()

    if results_df_2.empty:
        print("\n⚠️ Query 2 ran successfully but returned an empty result. No sales data for 'Tables' found.")
    else:
        display(results_df_2)

except Exception as e:
    print(f"\n❌ An error occurred with Query 2: {e}")

print("\n--- Note on Comparing Outcomes ---")
print("""
To compare the outcomes and see if your initial conclusion about 'Tables' holds true,
examine the results of these two alternative queries. Query 1 will show if the negative
profitability and high discounts for 'Tables' are consistent across different regions and
over different years. If you see variations (e.g., 'Tables' is profitable in certain
regions or years), it would nuance the idea that it's *always* negative due to discounts.
Query 2 will reveal if there's a correlation between specific ship modes and the profitability
or discount levels for 'Tables'. If one ship mode shows significantly worse profit and higher
discounts than others for 'Tables', it might suggest that the issue is more related to the logistics
or costs associated with that particular shipping method rather than the sub-category itself being
inherently unprofitable due to discounts across all orders.
""")

--- Results for Query 1 (Sliced by Region and Time) ---


,region,order_year,total_sales,total_profit,average_discount
0,Central,2014,7785.4780,-1424.3310,0.326667
1,Central,2015,6857.2600,-265.0939,0.207143
2,Central,2016,13922.9260,292.6211,0.205882
3,Central,2017,10589.3070,-2162.8466,0.292308
4,East,2014,10603.7040,-3537.8375,0.380000
5,East,2015,8884.8060,-2275.8641,0.373333
6,East,2016,7825.3280,-2306.7783,0.368182
7,East,2017,11825.9690,-2904.9002,0.373913
8,South,2014,9940.9445,1107.9902,0.113636
9,South,2015,7370.6745,-2171.3765,0.218750



--- Results for Query 2 (Control for Ship Mode) ---


,ship_mode,total_sales,total_profit,average_discount
0,Standard Class,124826.6615,-11910.0122,0.270526
1,Second Class,43693.7475,-3320.6799,0.248361
2,First Class,28800.7760,-1365.3665,0.240426
3,Same Day,9644.3470,-1129.4225,0.261905



--- Note on Comparing Outcomes ---

To compare the outcomes and see if your initial conclusion about 'Tables' holds true, 
examine the results of these two alternative queries. Query 1 will show if the negative 
profitability and high discounts for 'Tables' are consistent across different regions and 
over different years. If you see variations (e.g., 'Tables' is profitable in certain 
regions or years), it would nuance the idea that it's *always* negative due to discounts. 
Query 2 will reveal if there's a correlation between specific ship modes and the profitability 
or discount levels for 'Tables'. If one ship mode shows significantly worse profit and higher 
discounts than others for 'Tables', it might suggest that the issue is more related to the logistics 
or costs associated with that particular shipping method rather than the sub-category itself being 
inherently unprofitable due to discounts across all orders.



## Part H — Synthesis (DIVE: Extend)
**Aim:** Turn analysis into business‑ready insights.

### H1. Executive‑style summary
**Prompt:**
```
Act as a business strategist.
Based on the following metrics/figures (briefly summarize your results here), write a 4-sentence executive summary:
- 1 sentence: what changed and by how much
- 1 sentence: why it likely changed (drivers)
- 1 sentence: recommended action (who/what/when)
- 1 sentence: metric to monitor next
```

Based on our analysis, the 'Tables' sub-category consistently shows negative profitability overall, with significant profit losses observed across most regions and years, particularly when shipped via 'Standard Class'. This negative trend appears strongly correlated with higher average discounts applied to 'Tables' compared to other sub-categories and ship modes. To address this, the sales and logistics teams should immediately investigate the discounting strategy and associated costs specifically for 'Tables' shipped via 'Standard Class' by the end of the next quarter. A key metric to monitor going forward is the average profit margin for the 'Tables' sub-category, broken down by ship mode and region.

### H2. Convert final SQL into an automated job (optional)
**Prompt (use only after your SQL is final):**
```
Convert my final BigQuery SQL into a Python script that can run as a scheduled job from Colab or Cloud Functions.
Requirements:
- Use python‑bigquery client
- Parameterize date range
- Write results to a destination table `[YOUR_PROJECT].analytics.outputs_kpi`
- Add basic error handling & logging
Return one complete runnable script.
```

In [37]:
# Example of how to read from the destination table after the job runs
# (Optional - uncomment and run in a separate cell to verify)

# from google.cloud import bigquery
# import pandas as pd
#
# project_id = 'manifest-chain-471119-t8'
# destination_dataset_id = 'analytics'
# destination_table_id = 'outputs_kpi'
#
# try:
#     client = bigquery.Client(project=project_id)
#     query = f"SELECT * FROM `{project_id}.{destination_dataset_id}.{destination_table_id}` LIMIT 10;"
#     results_df_output = client.query(query).to_dataframe()
#     print("\n--- First 10 rows from destination table ---")
#     display(results_df_output)
# except Exception as e:
#     print(f"\n❌ An error occurred while reading from the destination table: {e}")

In [36]:
from google.cloud import bigquery
import logging
from datetime import datetime, timedelta

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Replace with your Google Cloud Project ID and dataset/table details
project_id = 'manifest-chain-471119-t8'
source_dataset_id = 'lab1_foundation'
source_table_id = 'superstore_clean'
destination_dataset_id = 'analytics' # Replace with your desired dataset
destination_table_id = 'outputs_kpi'  # Replace with your desired table

# Define the date range parameters (e.g., last 12 full months)
# This can be adjusted as needed, potentially passed as arguments in a real job
end_date = datetime.now().replace(day=1) - timedelta(days=1) # End of last month
start_date = (end_date.replace(day=1) - timedelta(days=365)).replace(day=1) # Start of 12 months prior


# Construct a BigQuery client object.
try:
    client = bigquery.Client(project=project_id)
    logging.info("BigQuery client created successfully.")
except Exception as e:
    logging.error(f"Failed to create BigQuery client: {e}")
    exit() # Exit if client creation fails

query_string = f"""
SELECT
    FORMAT_DATE('%Y-%m', DATE_TRUNC(order_date, MONTH)) AS year_month,
    SUM(sales) AS monthly_revenue
FROM
    `{project_id}.{source_dataset_id}.{source_table_id}`
WHERE
    order_date >= DATE('{start_date.strftime('%Y-%m-%d')}')
    AND order_date < DATE('{end_date.strftime('%Y-%m-%d')}') + INTERVAL 1 DAY
GROUP BY
    year_month
ORDER BY
    year_month ASC;
"""

logging.info(f"Executing query:\n{query_string}")

# Configure the job to write results to a destination table
job_config = bigquery.QueryJobConfig(destination=f"{project_id}.{destination_dataset_id}.{destination_table_id}")

# Set write disposition to overwrite the table if it exists
job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE


# Run the query
try:
    query_job = client.query(query_string, job_config=job_config)  # API request
    logging.info(f"Query job {query_job.job_id} submitted.")

    query_job.result()  # Waits for the query to finish
    logging.info(f"Query job {query_job.job_id} completed.")

    # You can optionally add code here to verify the results in the destination table
    logging.info(f"Results written to table: {project_id}.{destination_dataset_id}.{destination_table_id}")

except Exception as e:
    logging.error(f"An error occurred during query execution or writing results: {e}")

ERROR:root:An error occurred during query execution or writing results: 404 Not found: Dataset manifest-chain-471119-t8:analytics was not found in location US; reason: notFound, message: Not found: Dataset manifest-chain-471119-t8:analytics was not found in location US

Location: US
Job ID: f9ed2b71-aea3-4236-b79e-37bc6295564e



---
## Submission checklist
- [ ] Kept prompts precise and reproducible  
- [ ] Captured at least **one** CTE query and **one** window function query  
- [ ] Documented **two** validation attempts (counter‑queries or alternate slice)  
- [ ] Wrote a 4‑sentence executive summary based on results  
- [ ] (Optional) Converted final query into a scheduled job
---